In [9]:
import os
import json

import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [10]:
df= pd.read_csv('IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [11]:
df.shape

(50000, 2)

In [12]:
df.tail()


,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [13]:
df["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [14]:
df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

/tmp/ipython-input-1137712857.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [15]:
df["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [16]:
# split data into training data and test data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [17]:
print(train_df.shape)
print(test_df.shape)

(40000, 2)
(10000, 2)


In [18]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_df["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_df["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_df["review"]), maxlen=200)

In [20]:
Y_train = train_df["sentiment"]
Y_test = test_df["sentiment"]

LSTM - Long Short-Term Memory

In [21]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [24]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 207s 396ms/step - accuracy: 0.7150 - loss: 0.5360 - val_accuracy: 0.8549 - val_loss: 0.3510
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 197s 395ms/step - accuracy: 0.8581 - loss: 0.3448 - val_accuracy: 0.8518 - val_loss: 0.3433
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 208s 408ms/step - accuracy: 0.8777 - loss: 0.2983 - val_accuracy: 0.8315 - val_loss: 0.4012
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 257s 399ms/step - accuracy: 0.8761 - loss: 0.3110 - val_accuracy: 0.8789 - val_loss: 0.3020
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 208s 412ms/step - accuracy: 0.9115 - loss: 0.2259 - val_accuracy: 0.8786 - val_loss: 0.3296


In [25]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 116ms/step - accuracy: 0.8831 - loss: 0.3174
Test Loss: 0.31483525037765503
Test Accuracy: 0.8840000033378601


Predictive Model

In [26]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [30]:
# example usage
new_review = "This movie was bad."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
The sentiment of the review is: negative


In [31]:
# example usage
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
The sentiment of the review is: negative


In [1]:
import pickle